### Loading preprocessed corpus

In [ ]:
%run data_preprocessing.ipynb

In [ ]:
tweets_label = pd.read_csv('tweets_prepro.csv')

### Loading annotated corpus

In [ ]:
annotated_corpus = pd.read_csv('corpus_SexistContent.csv', sep='\t', header=None, names=['tweet_id', 'label'])

### Merging to recuperate labels of tweets

In [ ]:
tweets_label = pd.merge(tweets_label, annotated_corpus, on = 'tweet_id')

### Converting into a dataset for Pytorch to train model

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(tweets_label)
encoded_dataset = dataset.map(lambda tweets: tokenizer(tweets['text_clean']), batched=True)


### Splitting into train and test

### Training model

In [ ]:
#On utilise cette fonction car celle de sklearn ne marche pas pour les datasets.
#Seed = 42 pour qu'on ait le même découpage train/test à chaque fois
dataset_split = encoded_dataset.train_test_split(test_size=0.2, seed= 42)
train = dataset_split["train"]
test = dataset_split["test"]

In [ ]:
from transformers import CamembertForSequenceClassification, TrainingArguments, Trainer
model = CamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=2)

training_args = TrainingArguments(
    output_dir="results_of_model",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,    
    processing_class=tokenizer,          
)

trainer.train()